# Amazon Bedrock boto3 Setup

> **Pre-requisites**:
> - [Python 3.9](https://www.python.org/downloads/)

> **Setup environment**:
> - `python3.9 -m venv .venv`
> - `source .venv/bin/activate`
> - `pip install -r requirements.txt`
> ⚠️ You will see pip dependency errors, you can safely ignore these errors. ⚠️

- We will use AWS SDK directly to interact with AWS services.

### AWS Services

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

### agents

In [ ]:
%pip install --quiet \
    xmltodict==0.13.0  \
    duckduckgo-search  \
    yfinance  \
    pandas_datareader  \
    langchain_experimental \
    google-search-results

### entities

In [ ]:
%pip install --quiet beautifulsoup4


### image

In [ ]:
%pip install --quiet "pillow>=9.5,<10"


### gaurdrails

%%brew update
%%brew install gcc


#### Uninstall unwanted packages

In [ ]:
!python -m pip uninstall langchain -y
%pip uninstall langchain_experimental -y
%pip uninstall langchain-0.0.352 -y
%pip uninstall langchain_experimental -y

### nemoguardrails

In [ ]:
%pip install -qU --no-cache-dir nemoguardrails==0.5.0


%pip install -qU "faiss-cpu>=1.7,<2" \
                      "pypdf>=3.8,<4" \
                      "ipywidgets>=7,<8"

In [ ]:
%pip install -qU "langchain>=0.0.251" 
%pip install -qU "langchain_experimental>=0.0.30"

### Restart kernel

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")